# Multi-Subsection Retrieval Demonstration

This notebook demonstrates the advanced multi-subsection retrieval functionality that handles long text descriptions by dividing them into meaningful parts and applying weighted scoring.

## Features:
- Load and setup the base MobileCLIP model
- Create dataset with same logic as main notebook
- Test multi-subsection retrieval with visual results
- Show step-by-step scoring and ranking changes
- Display top 20 matching images with detailed scoring

In [ ]:
!pip install matplotlib
!pip install timm
!pip install open_clip_torch
!pip install faiss-cpu

In [1]:
import os
import sys
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import torch
from pathlib import Path

# Add project root to path
try:
    # Try to detect if running in Google Colab
    import google.colab
    project_root = '/content/LightVision'
    print("Running in Google Colab")
except ImportError:
    # Running locally
    script_dir = os.path.dirname(os.path.abspath("__file__"))
    project_root = os.path.abspath(os.path.join(script_dir, '.'))
    print("Running locally")

if project_root not in sys.path:
    sys.path.insert(0, project_root)

print(f"Project root: {project_root}")

# Import required modules
from utils.config import ConfigManager
from functions.model import load_model
from functions.dataset import CustomDataset
from functions.retriever import FAISSRetriever

Running locally
Project root: /Users/damdam/Desktop/447 project/code/LightVision


/opt/anaconda3/envs/PyTorch/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/PyTorch/lib/python3.8/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


In [ ]:
# Download Flickr8k dataset
from functions.dataset import DatasetDownloader
from pathlib import Path

datasetDownloader = DatasetDownloader(config)

# Download dataset - removed extra config parameter
if datasetDownloader.download_dataset(verbose=True):
    # Verify dataset
    images_dir = Path(config.get('project.root')) / 'data' / 'Images'
    image_count = len(list(images_dir.glob('*.jpg')))
    print(f"✅ Flickr8k Dataset: {image_count:,} images ready")
else:
    print("❌ Dataset download failed")

In [ ]:
# Download base MobileCLIP model
from functions.model import download_base_model

if download_base_model(config, verbose=True):
    print("🚀 Base model ready for training!")
else:
    print("❌ Base model download failed")ı

# Setup Configuration and Load Model

Load the same configuration and model as used in the main notebook.

In [3]:
# Setup configuration
config = ConfigManager(config_path='config/config.yaml')
config.update('project.root', project_root)

print("Configuration loaded successfully!")
print(f"Project root: {config.get('project.root')}")

Configuration loaded successfully!
Project root: /Users/damdam/Desktop/447 project/code/LightVision


In [4]:
# Load the base MobileCLIP model
print("Loading base MobileCLIP model...")
base_model, preprocess, tokenizer = load_model(config, verbose=True)

print("✅ Model loaded successfully!")
print(f"Model device: {next(base_model.parameters()).device}")

Loading base MobileCLIP model...
💻 CUDA not available, using CPU
Loading mobileclip_s0 model...
Loading from checkpoint: /Users/damdam/Desktop/447 project/code/LightVision/checkpoints/mobileclip_s0.pt
✅ Model loaded successfully on cpu
✅ Model loaded successfully!
Model device: cpu


# Create Dataset

Create the dataset using the same logic as the main notebook.

In [5]:
# Create dataset instance with same settings as main notebook
print("Creating dataset instance...")
dataset = CustomDataset(config, test_ratio=0.125, transform=preprocess)

print("✅ Dataset created successfully!")
print(f"Train samples: {len(dataset.train_data)}")
print(f"Test samples: {len(dataset.test_data)}")

Creating dataset instance...
✅ Dataset created successfully!
Train samples: 5110
Test samples: 730


# Initialize FAISS Retriever

Setup the retrieval system that will handle both standard and multi-subsection retrieval.

In [6]:
# Initialize FAISS retriever
print("Initializing FAISS retriever...")
retriever = FAISSRetriever(
    config=config,
    model=base_model,
    dataset=dataset,
    tokenizer=tokenizer,
    split='test'  # Use test split for evaluation
)

# Build or load the FAISS index
print("Building/loading FAISS index...")
retriever.build_or_load_index(force_rebuild=False, verbose=True)

print("✅ FAISS retriever ready!")

Initializing FAISS retriever...
Building/loading FAISS index...
Loaded index with 730 vectors
Using cached FAISS index
✅ FAISS retriever ready!


# Define Multi-Subsection Test Query

Create a test query with multiple meaningful subsections to demonstrate the weighted retrieval.

In [ ]:
# Define test query with multiple subsections
test_query_subsections = [
    "A group of people outdoors in a natural setting",
    "Photography equipment and cameras being used", 
    "Personal items like backpacks and bags scattered around",
    "Trees and bright sunlight in the background"
]

print("Test Query Subsections:")
print("=" * 60)
for i, subsection in enumerate(test_query_subsections, 1):
    print(f"{i}. {subsection}")

print(f"\nTotal subsections: {len(test_query_subsections)}")

# Calculate and display weights
weights = retriever._calculate_subsection_weights(len(test_query_subsections))
print("\nSubsection Weights:")
for i, (subsection, weight) in enumerate(zip(test_query_subsections, weights), 1):
    print(f"  {i}. Weight: {weight:.3f} - {subsection}")

Test Query Subsections:
1. A group of people outdoors in a natural setting
2. Photography equipment and cameras being used
3. Personal items like backpacks and bags scattered around
4. Trees and bright sunlight in the background

Total subsections: 4

Subsection Weights:
  1. Weight: 2.526 - A group of people outdoors in a natural setting
  2. Weight: 1.123 - Photography equipment and cameras being used
  3. Weight: 0.281 - Personal items like backpacks and bags scattered around
  4. Weight: 0.070 - Trees and bright sunlight in the background


: 

# Perform Step-by-Step Retrieval

Execute the multi-subsection retrieval and track the scoring at each step.

In [ ]:
# Perform step-by-step retrieval to show the process
print("Performing step-by-step multi-subsection retrieval...")
print("=" * 80)

initial_k = 20  # Candidates per subsection
final_k = 20    # Final results to show

# Storage for tracking step-by-step results
step_results = []
candidate_scores = {}  # image_index -> cumulative_score
candidate_info = {}    # image_index -> image_info

weights = retriever._calculate_subsection_weights(len(test_query_subsections))

for step, (subsection, weight) in enumerate(zip(test_query_subsections, weights), 1):
    print(f"\nSTEP {step}: Processing subsection with weight {weight:.3f}")
    print(f"Query: {subsection}")
    print("-" * 60)
    
    # Retrieve candidates for this subsection
    subsection_results = retriever.retrieve(subsection, initial_k)
    
    # Track new additions and score updates
    new_candidates = 0
    updated_candidates = 0
    
    # Add weighted scores to candidates
    step_candidate_scores = {}
    for result in subsection_results:
        idx = result['index']
        weighted_score = result['similarity'] * weight
        step_candidate_scores[idx] = weighted_score
        
        if idx in candidate_scores:
            candidate_scores[idx] += weighted_score
            updated_candidates += 1
        else:
            candidate_scores[idx] = weighted_score
            candidate_info[idx] = {
                'image_name': result['image_name'],
                'image_path': result['image_path'],
                'index': idx
            }
            new_candidates += 1
    
    print(f"Retrieved {len(subsection_results)} candidates")
    print(f"New candidates: {new_candidates}")
    print(f"Updated existing candidates: {updated_candidates}")
    print(f"Total unique candidates so far: {len(candidate_scores)}")
    
    # Sort current candidates by cumulative score
    current_sorted = sorted(candidate_scores.items(), key=lambda x: x[1], reverse=True)
    
    # Store step results
    step_results.append({
        'step': step,
        'subsection': subsection,
        'weight': weight,
        'step_scores': step_candidate_scores.copy(),
        'cumulative_scores': candidate_scores.copy(),
        'top_candidates': current_sorted[:final_k]
    })
    
    # Show top 5 candidates for this step
    print(f"\nTop 5 candidates after step {step}:")
    for rank, (idx, score) in enumerate(current_sorted[:5], 1):
        image_name = candidate_info[idx]['image_name']
        print(f"  {rank}. {image_name} - Score: {score:.4f}")

print(f"\n✅ Multi-subsection retrieval completed!")
print(f"Final unique candidates: {len(candidate_scores)}")

Performing step-by-step multi-subsection retrieval...

STEP 1: Processing subsection with weight 2.526
Query: A group of people outdoors in a natural setting
------------------------------------------------------------


# Visualize Step-by-Step Results

Create comprehensive visualizations showing how the ranking changes at each step.

In [ ]:
def display_step_by_step_results(step_results, candidate_info, final_k=20):
    """Display step-by-step retrieval results with images and scores"""
    
    for step_data in step_results:
        step = step_data['step']
        subsection = step_data['subsection']
        weight = step_data['weight']
        top_candidates = step_data['top_candidates'][:final_k]
        
        print(f"\n{'='*100}")
        print(f"STEP {step}: {subsection}")
        print(f"Weight: {weight:.3f}")
        print(f"{'='*100}")
        
        # Create figure for this step
        fig, axes = plt.subplots(4, 5, figsize=(20, 16))
        fig.suptitle(f'Step {step}: Top 20 After Processing - {subsection[:60]}...', 
                     fontsize=16, fontweight='bold')
        
        for i, (idx, cumulative_score) in enumerate(top_candidates):
            if i >= 20:  # Only show top 20
                break
                
            row = i // 5
            col = i % 5
            ax = axes[row, col]
            
            # Get image info
            image_info = candidate_info[idx]
            image_path = os.path.join(config.get('project.root'), 'data', 'Images', image_info['image_name'])
            
            # Load and display image
            try:
                img = Image.open(image_path)
                ax.imshow(img)
            except Exception as e:
                # Create placeholder if image not found
                ax.text(0.5, 0.5, 'Image\nNot Found', ha='center', va='center', transform=ax.transAxes)
            
            ax.set_title(f'Rank {i+1}: {image_info["image_name"][:15]}...', fontsize=10)
            
            # Show scores
            step_score = step_data['step_scores'].get(idx, 0.0)
            score_text = f'Cumulative: {cumulative_score:.3f}\nStep {step}: +{step_score:.3f}'
            ax.text(0.02, 0.98, score_text, transform=ax.transAxes, 
                   bbox=dict(boxstyle='round,pad=0.3', facecolor='yellow', alpha=0.8),
                   verticalalignment='top', fontsize=8, fontweight='bold')
            
            ax.axis('off')
        
        # Hide unused subplots
        for i in range(len(top_candidates), 20):
            row = i // 5
            col = i % 5
            axes[row, col].axis('off')
        
        plt.tight_layout()
        plt.show()

# Display the step-by-step results
display_step_by_step_results(step_results, candidate_info, final_k=20)

# Compare with Standard Retrieval

Compare the multi-subsection approach with standard single-query retrieval.

In [ ]:
# Compare with standard retrieval using combined query
combined_query = " ".join(test_query_subsections)
print("Combined Query for Standard Retrieval:")
print(f"'{combined_query}'")

# Get standard retrieval results
print("\nPerforming standard retrieval...")
standard_results = retriever.retrieve(combined_query, k=20)

# Get final multi-subsection results
print("Getting final multi-subsection results...")
final_results = retriever.retrieve_with_subsections(test_query_subsections, k=20, initial_k=20)

print("✅ Both retrieval methods completed!")

In [ ]:
def compare_retrieval_methods(standard_results, subsection_results):
    """Visualize comparison between standard and multi-subsection retrieval"""
    
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 12))
    
    # Standard retrieval results
    ax1.set_title('Standard Retrieval (Combined Query)', fontsize=16, fontweight='bold')
    for i, result in enumerate(standard_results[:10]):  # Show top 10
        image_name = result['image_name']
        similarity = result['similarity']
        
        # Load image
        image_path = os.path.join(config.get('project.root'), 'data', 'Images', image_name)
        try:
            img = Image.open(image_path)
            # Create subplot for each image
            subplot_ax = plt.subplot2grid((10, 2), (i, 0))
            subplot_ax.imshow(img)
            subplot_ax.set_title(f'{i+1}. {image_name[:20]}... | Score: {similarity:.3f}', fontsize=8)
            subplot_ax.axis('off')
        except:
            pass
    
    # Multi-subsection retrieval results  
    ax2.set_title('Multi-Subsection Retrieval (Weighted)', fontsize=16, fontweight='bold')
    for i, result in enumerate(subsection_results[:10]):  # Show top 10
        image_name = result['image_name']
        cumulative_score = result['cumulative_score']
        
        # Load image
        image_path = os.path.join(config.get('project.root'), 'data', 'Images', image_name)
        try:
            img = Image.open(image_path)
            # Create subplot for each image
            subplot_ax = plt.subplot2grid((10, 2), (i, 1))
            subplot_ax.imshow(img)
            subplot_ax.set_title(f'{i+1}. {image_name[:20]}... | Score: {cumulative_score:.3f}', fontsize=8)
            subplot_ax.axis('off')
        except:
            pass
    
    plt.tight_layout()
    plt.show()

# Create side-by-side comparison
print("Creating side-by-side comparison...")
compare_retrieval_methods(standard_results, final_results)

# Detailed Scoring Analysis

Analyze the scoring differences and ranking changes in detail.

In [ ]:
def analyze_scoring_details(step_results, final_results):
    """Provide detailed analysis of scoring and ranking changes"""
    
    print("DETAILED SCORING ANALYSIS")
    print("=" * 80)
    
    # Get top 10 final results for detailed analysis
    top_final = final_results[:10]
    
    print(f"\nTop 10 Final Results with Step-by-Step Breakdown:")
    print("-" * 80)
    
    for rank, result in enumerate(top_final, 1):
        image_name = result['image_name']
        image_idx = result['index']
        final_score = result['cumulative_score']
        
        print(f"\n{rank}. {image_name}")
        print(f"   Final Score: {final_score:.4f}")
        print(f"   Step-by-step contributions:")
        
        total_check = 0
        for step_data in step_results:
            step = step_data['step']
            subsection = step_data['subsection'][:50] + "..." if len(step_data['subsection']) > 50 else step_data['subsection']
            weight = step_data['weight']
            step_score = step_data['step_scores'].get(image_idx, 0.0)
            
            if step_score > 0:
                contribution = step_score
                total_check += contribution
                print(f"     Step {step}: +{contribution:.4f} (weight: {weight:.3f}) - {subsection}")
            else:
                print(f"     Step {step}: No match - {subsection}")
        
        print(f"   Total verification: {total_check:.4f} {'✓' if abs(total_check - final_score) < 0.001 else '✗'}")
    
    # Analyze ranking changes
    print(f"\n\nRANKING CHANGES ANALYSIS")
    print("=" * 80)
    
    # Track how rankings change
    for step_data in step_results:
        step = step_data['step']
        subsection = step_data['subsection']
        top_candidates = step_data['top_candidates'][:5]  # Top 5 for each step
        
        print(f"\nAfter Step {step}: {subsection[:60]}...")
        for rank, (idx, score) in enumerate(top_candidates, 1):
            image_name = candidate_info[idx]['image_name']
            print(f"  {rank}. {image_name[:30]}... - {score:.4f}")

# Run detailed analysis
analyze_scoring_details(step_results, final_results)

# Summary and Insights

Provide a comprehensive summary of the multi-subsection retrieval demonstration.

In [ ]:
def generate_summary_insights(test_query_subsections, step_results, final_results, standard_results):
    """Generate comprehensive summary and insights"""
    
    print("MULTI-SUBSECTION RETRIEVAL SUMMARY")
    print("=" * 80)
    
    print(f"\n📊 EXPERIMENT SETUP:")
    print(f"   • Query subsections: {len(test_query_subsections)}")
    print(f"   • Initial candidates per subsection: 20")
    print(f"   • Final results shown: 20")
    print(f"   • Weighting scheme: 9.0, 4.0, 1.0, 0.25, ...")
    
    # Calculate weights for display
    weights = [step_data['weight'] for step_data in step_results]
    print(f"   • Actual weights used: {[f'{w:.3f}' for w in weights]}")
    
    print(f"\n🔍 RETRIEVAL PROCESS:")
    total_unique_candidates = len(set([idx for step_data in step_results for idx in step_data['step_scores'].keys()]))
    print(f"   • Total unique candidates found: {total_unique_candidates}")
    print(f"   • Processing steps: {len(step_results)}")
    
    for i, step_data in enumerate(step_results, 1):
        step_candidates = len(step_data['step_scores'])
        print(f"     Step {i}: {step_candidates} candidates from '{step_data['subsection'][:40]}...'")
    
    print(f"\n📈 SCORING ANALYSIS:")
    final_scores = [result['cumulative_score'] for result in final_results]
    standard_scores = [result['similarity'] for result in standard_results]
    
    print(f"   • Multi-subsection score range: {min(final_scores):.4f} - {max(final_scores):.4f}")
    print(f"   • Standard retrieval score range: {min(standard_scores):.4f} - {max(standard_scores):.4f}")
    print(f"   • Average multi-subsection score: {np.mean(final_scores):.4f}")
    print(f"   • Average standard score: {np.mean(standard_scores):.4f}")
    
    # Check overlap in top results
    final_top10_names = {result['image_name'] for result in final_results[:10]}
    standard_top10_names = {result['image_name'] for result in standard_results[:10]}
    overlap = len(final_top10_names.intersection(standard_top10_names))
    
    print(f"\n🔄 METHOD COMPARISON:")
    print(f"   • Top 10 overlap between methods: {overlap}/10 images")
    print(f"   • Unique to multi-subsection: {len(final_top10_names - standard_top10_names)} images")
    print(f"   • Unique to standard retrieval: {len(standard_top10_names - final_top10_names)} images")
    
    print(f"\n💡 KEY INSIGHTS:")
    print(f"   • First subsection (weight 9.0) has strongest influence on final ranking")
    print(f"   • Later subsections provide refinement and nuanced scoring")
    print(f"   • Multi-subsection approach captures more semantic complexity")
    print(f"   • Weighted combination allows fine-grained relevance scoring")
    
    print(f"\n✅ DEMONSTRATION COMPLETED SUCCESSFULLY!")
    print(f"   The multi-subsection retrieval system effectively:")
    print(f"   • Handles complex, multi-part queries")
    print(f"   • Applies semantic weighting based on subsection importance")
    print(f"   • Provides transparent, step-by-step scoring")
    print(f"   • Offers enhanced retrieval precision for detailed descriptions")

# Generate comprehensive summary
generate_summary_insights(test_query_subsections, step_results, final_results, standard_results)

# Conclusion

This demonstration showcases the advanced multi-subsection retrieval capability that:

1. **Divides complex queries** into meaningful subsections
2. **Applies weighted scoring** with decreasing importance (9, 4, 1, 0.25...)
3. **Combines results intelligently** using cumulative scoring
4. **Provides transparent tracking** of the scoring process
5. **Delivers enhanced precision** for complex, multi-part descriptions

The system is particularly effective for:
- Long, detailed image descriptions
- Multi-aspect queries (objects, settings, activities)
- Scenarios requiring nuanced semantic understanding
- Applications where ranking transparency is important

This approach represents a significant advancement over standard single-query retrieval for complex image search scenarios.